## Dağıtım Ağı Modelleme

<img src="image2.png" alt="Drawing" style="width: 600px;"/>

### Problem tanımı

Uluslararası bir üretim şirketinin Lojistik Ağı Yöneticisi olarak, taşıma maliyetlerinde son zamanlarda görülen artışı ve gelecekteki talep tahminlerini göz önünde bulundurarak şirketin lojistik ağını önümüzdeki 5 yıl için yeniden tasarlamak istiyorsunuz.

#### Ağ yapısı
* Brezilya, ABD, Hindistan, Japonya, Almanya olmak üzere 5 ülkede üretim ve satış yapıyoruz
* Her ülkeden 2 tip üretim tesisi var: düşük ve yüksek kapasite
* Taşıma maliyetleri (USD/konteyner)
* Piyasa talebi (Adet/yıl)

#### Maliyetler
* Sabit maliyetler (makina/ekipman, kira, vb.)
* Üretim değişken maliyetleri (üretim hattı operatörleri, hammadde, vb.)
* Taşıma değişken maliyetleri (1 konteynır = 1000 adet)

In [467]:
import pandas as pd
from pulp import *

In [483]:
fixed_costs = pd.read_excel('input_data.xlsx', 'fixed_costs',index_col = 0)
#fixed_costs.loc['Hindistan','Yuksek'] *=2
fixed_costs

,Dusuk,Yuksek
Sabit mal. ($/ay),,
ABD,6500000,9500000
Almanya,4980000,7270000
Japonya,6230000,9100000
Brezilya,3230000,4730000
Hindistan,2110000,3080000


In [484]:
# Import Costs
manvar_costs = pd.read_excel('input_data.xlsx', 'manvar_costs', index_col = 0)
manvar_costs

,ABD,Almanya,Japonya,Brezilya,Hindistan
Değişken mal. ($/adet),,,,,
ABD,12,12,12,12,12
Almanya,13,13,13,13,13
Japonya,10,10,10,10,10
Brezilya,8,8,8,8,8
Hindistan,5,5,5,5,5


In [486]:
# Import Costs
freight_costs = pd.read_excel('input_data.xlsx', 'freight_costs', index_col = 0)/1000
#freight_costs *=7
freight_costs

,ABD,Almanya,Japonya,Brezilya,Hindistan
Taşıma mal. ($/konteynır),,,,,
ABD,0.000,1.75,1.100,2.300,1.254
Almanya,1.905,0.00,1.231,2.892,1.439
Japonya,2.200,3.25,0.000,6.230,2.050
Brezilya,2.350,3.15,4.000,0.000,4.250
Hindistan,1.950,2.20,3.500,4.200,0.000


In [487]:
# Variable Costs
var_cost = freight_costs + manvar_costs 
var_cost

,ABD,Almanya,Japonya,Brezilya,Hindistan
Taşıma mal. ($/konteynır),,,,,
ABD,12.000,13.75,13.100,14.300,13.254
Almanya,14.905,13.00,14.231,15.892,14.439
Japonya,12.200,13.25,10.000,16.230,12.050
Brezilya,10.350,11.15,12.000,8.000,12.250
Hindistan,6.950,7.20,8.500,9.200,5.000


#### Fixed Costs

#### Üretim kapasiteleri

In [472]:
# Two types of plants: Low Capacity and High Capacity Plant
kapasite = pd.read_excel('input_data.xlsx', 'capacities', index_col = 0)
#kapasite.loc['Hindistan','Yuksek'] *=2
kapasite

,Dusuk,Yuksek
Capacity (Units/month),,
ABD,500000,1500000
Almanya,500000,1500000
Japonya,500000,1500000
Brezilya,500000,1500000
Hindistan,500000,1500000


#### Piyasa talepleri

In [473]:
# -- Demand
talep = pd.read_excel('input_data.xlsx', 'demands', index_col = 0)
talep

,Talep
(Units/month),
ABD,2800000
Almanya,90000
Japonya,1700000
Brezilya,145000
Hindistan,160000


In [474]:
# Karar değişkenlerini tanımla
lokasyon = ['ABD', 'Almanya', 'Japonya', 'Brezilya', 'Hindistan']
tip = ['Dusuk', 'Yuksek']

In [475]:
# Modeli tanımla
model = LpProblem('DagitimAgiModeli',LpMinimize)

In [476]:
# Karar değişkenlerini oluştur
x = LpVariable.dicts('transfer', [(i,j) for i in lokasyon for j in lokasyon], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('kur', [(i,s) for i in lokasyon for s in tip], cat='Binary')

In [477]:
# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] for s in tip for i in lokasyon]) 
          + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in lokasyon for j in lokasyon]))

In [ ]:
model

In [479]:
# Kısıtları ekle
for j in lokasyon:
    model += lpSum([x[(i, j)] for i in lokasyon]) == talep.loc[j,'Talep']
for i in lokasyon:
    model += lpSum([x[(i, j)] for j in lokasyon]) <= lpSum([kapasite.loc[i,s]*y[(i,s)] for s in tip])

In [480]:
# Define logical constraint: Add a logical constraint so that if the high capacity plant in USA is open, 
# then a low capacity plant in Germany is also opened.
# model += y[('USA','High_Cap')] <= y[('Germany','Low_Cap')]                                                       
                                                       
# Modeli coz
model.solve()
print('Toplam maliyet = {:,} ($/Month)'.format(int(value(model.objective))))
print("Durum: {}".format(LpStatus[model.status]))

Toplam maliyet = 92,671,000 ($/Month)
Durum: Optimal


In [481]:
# Dictionary
dict_plant = {}
dict_prod = {}
print('Uretim yapacak fabrikalar ve tipleri')
for v in model.variables():
    if ('kur' in v.name) and (v.varValue == 1):
        name = v.name.replace('kur_', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        #p_name = name
        print(name)#, '=', capacity.loc[p_name,])#, "=", v.varValue)
print('\nUretim/transfer miktarlari')
for v in model.variables():
    if ('transfer' in v.name) and (v.varValue > 0):
        name = v.name.replace('transfer_', '').replace('_', '')
        dict_prod[name] = v.varValue
        print(name, "=", '{:,} adet'.format(int(value(v.varValue))))
#capacities

Uretim yapacak fabrikalar ve tipleri
('ABD','Yuksek')
('Brezilya','Dusuk')
('Hindistan','Yuksek')
('Japonya','Yuksek')

Uretim/transfer miktarlari
('ABD','ABD') = 1,300,000 adet
('ABD','Japonya') = 200,000 adet
('Brezilya','ABD') = 250,000 adet
('Brezilya','Brezilya') = 145,000 adet
('Hindistan','ABD') = 1,250,000 adet
('Hindistan','Almanya') = 90,000 adet
('Hindistan','Hindistan') = 160,000 adet
('Japonya','Japonya') = 1,500,000 adet


#### Senaryo A: Üretimi işgücü maliyeti düşük olan bölgeye kaydırma
Hindistan'daki yüksek kapasiteli fabrikanın kapasitesini 2 katına çıkarırsak ne olur? (Sabit maliyetin de 2 katına çıktığını varsayalım.)
* Brazil plant is still producing for the local market and the USA
* India plants produce for all countries except Brazil
* Japan does not produce locally anymore.
* Total Costs = 51,352,000 \$/Month (19.4\% less than Scenario 1)

#### Senaryo B: Konteynır kısıtı nedeniyle artan taşıma maliyetleri
Konteynır kısıtı nedeniyle taşıma maliyetleri 5 katına çıkarsa ne olur?
* Brazil is producing for the local market only
* The USA started to produce for the local market and Japan
* India closed its low capacity factory
* Japan starts to produce for its local market
* Because of their limited production capacity, Japan and the USA still rely on the Indian plant
* Total Costs = 92,981,000 $/Month